# Capstone Project: Slogan Classifier and Generator

In this capstone project you will train a Long Short-Term Memory (LSTM) model to generate slogans for businesses based on their industry, and also train a classifier to predict the industry based on a given slogan.

In [92]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import spacy # available on Google Colab
from sklearn.model_selection import train_test_split
from google.colab import drive

## Loading and viewing the dataset

- Load the slogan dataset into a variable called data.
- Extract relevant columns in a variable called df.
- Handle missing values.

Do **not** change the column names.

If you are using Google Colab you will need mount your Google Drive as follows:  
`from google.colab import drive`  
`drive.mount('/content/drive')`  

The path you use when loading your data will look something like this if you are using your Google Drive:  
"/content/drive/MyDrive/Colab Notebooks/slogan-valid.csv"

In [93]:
# Load data from your local machine or from your Google Drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/slogan-valid.csv')

# Extract relevant columns
df = data[['industry', 'output']]

# Remove any rows that have missing values
df = df.dropna()
df.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,industry,output
0,computer hardware,Taking Care of Small Business Technology
1,"health, wellness and fitness",Build World-Class Recreation Programs
2,internet,Most Powerful Lead Generation Software for Mar...
3,internet,Hire quality freelancers for your job
4,financial services,"Financial Advisers Norwich, Norfolk"


## Data Preprocessing

Since we are working with textual data, we need software that understands natural language. For this, we'll use a library for processing text called **spaCy**. Using spaCy, we'll break the text into smaller units called tokens that are easier for the machine to process. This process is called **tokenisation**. We'll also convert all text to lowercase and remove punctuation because this information is not necessary for our models. Run the code below, and your dataframe (df) will gain a new column called **'processed_slogan'** which contains the preprocessed text.


In [94]:
# Load spaCy model for text processing
nlp = spacy.load("en_core_web_sm")

# Define text preprocessing function
def preprocess_text(text):
    text_lower = text.lower()
    doc = nlp(text_lower)

    processed_tokens = []

    for token in doc:
        if not token.is_punct:
            processed_tokens.append(token.text)

    return " ".join(processed_tokens)

df["processed_slogan"] = df["output"].apply(preprocess_text)

df.head()

,industry,output,processed_slogan
0,computer hardware,Taking Care of Small Business Technology,taking care of small business technology
1,"health, wellness and fitness",Build World-Class Recreation Programs,build world class recreation programs
2,internet,Most Powerful Lead Generation Software for Mar...,most powerful lead generation software for mar...
3,internet,Hire quality freelancers for your job,hire quality freelancers for your job
4,financial services,"Financial Advisers Norwich, Norfolk",financial advisers norwich norfolk


We want our model to generate **industry-specific** slogans. If we use the 'processed_slogan' column as it is, we'll be leaving out crucial context - the industries of the companies behind those slogans. To fix this, we'll create a new **'modified_slogan'** column that adds the industry name to the front of processed slogan.  

For example:  

> industry = 'computer hardware'  
processed_slogan = 'taking care of small business technology'  
modified_slogan = 'computer hardware taking care of small business technology'

Write code in the cell below to achieve this.

In [95]:
# Add industry name to each slogan
df['modified_slogan'] = df['industry'] + ' ' + df['processed_slogan']
df.head()

,industry,output,processed_slogan,modified_slogan
0,computer hardware,Taking Care of Small Business Technology,taking care of small business technology,computer hardware taking care of small busines...
1,"health, wellness and fitness",Build World-Class Recreation Programs,build world class recreation programs,"health, wellness and fitness build world class..."
2,internet,Most Powerful Lead Generation Software for Mar...,most powerful lead generation software for mar...,internet most powerful lead generation softwar...
3,internet,Hire quality freelancers for your job,hire quality freelancers for your job,internet hire quality freelancers for your job
4,financial services,"Financial Advisers Norwich, Norfolk",financial advisers norwich norfolk,financial services financial advisers norwich ...


Now we need to get data to train our model. We have textual data which we will need to represent numerically for our model to learn from it.  
The code below does the following:
1. Tokenizes a dataset of slogans.
2. Converts words to numerical indices.
3. Creates input sequences using the numerical indices.  

Here's how it works. From the 'modified_slogan' column, we take the slogan "computer hardware taking care of small business technology". The tokenisation process will convert words into their corresponding indices:  

<center>

| Word         | Token Index |
|-------------|-------|
| "computer"  | 1     |
| "hardware"  | 2     |
| "taking"    | 3     |
| "care"      | 4     |
| "of"        | 5     |
| "small"     | 6     |
| "business"  | 7     |
| "technology"| 8     |

</center>

So the tokenized list is:

<center>
[1, 2, 3, 4, 5, 6, 7, 8]
</center>

When creating input sequences for training, the loop generates progressively longer sequences.

<center>

| Token Index Sequence               | Corresponding Slogan                                 |
|------------------------------|-----------------------------------------------------|
| [1, 2]                       | "computer hardware"                                |
| [1, 2, 3]                    | "computer hardware taking"                        |
| [1, 2, 3, 4]                 | "computer hardware taking care"                   |
| [1, 2, 3, 4, 5]              | "computer hardware taking care of"                |
| [1, 2, 3, 4, 5, 6]           | "computer hardware taking care of small"          |
| [1, 2, 3, 4, 5, 6, 7]        | "computer hardware taking care of small business" |
| [1, 2, 3, 4, 5, 6, 7, 8]     | "computer hardware taking care of small business technology" |

</center>

Instead of training the model on only **complete slogans**, we provide partial phrases which will help the model learn how words connect over time. This will make it better at predicting the next word when generating slogans.  

Run the cell block below to generate the input sequences. Be sure to read the comments to understand what the code is doing.

In [96]:
'''** Clean up comments'''

# Tokenizer to convert words into numerical values tokens
tokenizer = Tokenizer()

# Tokenizer learns words in dataset
tokenizer.fit_on_texts(df["modified_slogan"])

# Total number of unique words in learned vocabulary
total_words = len(tokenizer.word_index) + 1

# Dictionary mapping words to its numerical index: index based on frequency i.e., more freq => lower index
tokenizer.word_index

# Creating input sequences
# Initialise list to store the input sequences
input_sequences = []

# Iterate over processed slogans
for line in df["modified_slogan"]:

    # Convert slogans to token sequences
    token_list = tokenizer.texts_to_sequences([line])[0] # returns list containing list of words indices; extracting inner list [0]

    # token_list is a list of tokenized word INDICES
    # Building list of progressively longer input sequences for better training
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

The input sequences created above are of **varying lengths**, which will be a problem when training our LSTM model. LSTMs require input sequences of **equal length**. So, we need to **pad** shorter sequences by **prepending zeros** until they match the length of the longest sequence.  

For example, if the longest sequence has **10 tokens**, our padded sequences will look like this:

<center>

| Input Sequence                     | Padded Sequence                         |
|-------------------------------------|-----------------------------------------|
| [1, 2]                              | [0, 0, 0, 0, 0, 0, 0, 0, 1, 2]         |
| [1, 2, 3]                           | [0, 0, 0, 0, 0, 0, 0, 1, 2, 3]         |
| [1, 2, 3, 4]                        | [0, 0, 0, 0, 0, 0, 1, 2, 3, 4]         |
| [1, 2, 3, 4, 5]                     | [0, 0, 0, 0, 0, 1, 2, 3, 4, 5]         |
| [1, 2, 3, 4, 5, 6]                  | [0, 0, 0, 0, 1, 2, 3, 4, 5, 6]         |
| [1, 2, 3, 4, 5, 6, 7]               | [0, 0, 0, 1, 2, 3, 4, 5, 6, 7]         |
| [1, 2, 3, 4, 5, 6, 7, 8]            | [0, 0, 1, 2, 3, 4, 5, 6, 7, 8]         |

</center>

In the cell below, write code that **finds the length of the longest sequence** in **input_sequences** and stores this value in a variable named **max_seq_len**.


In [97]:
max_seq_len = max([len(sequence) for sequence in input_sequences])

print(f"Maximal sequence length: {max_seq_len}")

Maximal sequence length: 15


Run the cell below to pad the input sequences so they are all the same length as **max_seq_length**.

In [98]:
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding="pre")

## Training Data for Slogan Generator

The input sequences generated will be used as our training data. Our LSTM needs to learn how to predict the **next word** in a sequence.  

The inputs for our model will be the input sequences **excluding the last token index** and the outputs will be the **last token index**.  

As an example, let us use the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7, 8] and say it corresponds to the slogan "computer hardware taking care of small business technology". When training the model:

> Our input **x** will be the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7] corresponding to "computer hardware taking care of small".  
> Our output **y** will be [8] which corresponds to "business".  

In the code cell below, use `input_sequences` to create the following two variables:
1. **X_gen** which contains the input sequences excluding the last token index.
2. **y_gen** which contains the last token index of the input sequence.

In [99]:
# Split sequences into X and Y
X_gen = input_sequences[:, :-1]
Y_gen = input_sequences[:, -1]

print(f"X_gen shape: {X_gen.shape}")
print(f"Y_gen shape: {Y_gen.shape}")


X_gen shape: (34736, 14)
Y_gen shape: (34736,)


The model will output the next word of a sequence over a probability distribution. We need to encode our output variable for this to be possible.

In the code cell below, write code that will apply one-hot encoding to **y_gen** using `tf.keras.utils.to_categorical()`. **Maintain the same variable name**.  

*Hint: set the `num_classes` (number of classes) parameter to the total number of unique words in the learned vocabulary. You can access this value through a variable that was created when generating input sequences earlier.*

In [100]:
# Convert y_gen to one-hot coding format
y_gen = tf.keras.utils.to_categorical(Y_gen, num_classes=total_words)

print(f"y_gen shape: {y_gen.shape}")

y_gen shape: (34736, 6046)


## Slogan Generator Architecture

In the code cell that follows, configure the LSTM following these steps:

1. Create a sequential model using `tf.keras.models.Sequential()`. This model will have an embedding layer, two LSTM layers, and a dense output layer.
2. Add an embedding layer that converts words into dense vector representations. This layer should:
> *   Have `total_words`as the vocabulary size.
> *   Use 100 as an embedding dimension.
> *   Takes an input length of `max_seq_len - 1` (excludes the target word).
3. Add two LSTM layers.
> *   The first LSTM layer should have 150 **and** set `return_sequences` to `True`.
> *   The second LSTM layer should have 100 units.
4. Add a dense output layer which:
> *   Uses `total_words` as the number of units (one for each word in the vocabulary).
> *   Uses a softmax activation function.
5. Use `Sequential` to put everything together in the correct order to complete the architecture of the LSTM model called **gen_model**.

In [101]:
# Creating model structure
gen_model = Sequential([
    # Embedding layer
    Embedding(input_dim=total_words, output_dim=100, input_length=max_seq_len-1),
    # LSTM layers
    LSTM(150, return_sequences=True),
    LSTM(100),
    # Dense layer
    Dense(total_words, activation='softmax')
])

# Model structure
gen_model.summary()

# Configure model learning
gen_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Slogan Generation

In the code cell below, fit the compiled model on the inputs and outputs, setting the **number of epochs to 50**.

In [103]:
# Train model
history = gen_model.fit(X_gen, y_gen, epochs=50, verbose=1)

Epoch 1/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 79s 73ms/step - accuracy: 0.1104 - loss: 6.2666
Epoch 2/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 77s 71ms/step - accuracy: 0.1386 - loss: 5.9704
Epoch 3/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 77s 71ms/step - accuracy: 0.1685 - loss: 5.7035
Epoch 4/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 79s 73ms/step - accuracy: 0.1956 - loss: 5.4193
Epoch 5/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 77s 71ms/step - accuracy: 0.2060 - loss: 5.2162
Epoch 6/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 82s 75ms/step - accuracy: 0.2223 - loss: 4.9719
Epoch 7/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 139s 72ms/step - accuracy: 0.2383 - loss: 4.7303
Epoch 8/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 81s 71ms/step - accuracy: 0.2502 - loss: 4.5213
Epoch 9/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 81s 75ms/step - accuracy: 0.2621 - loss: 4.3105
Epoch 10/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 80s 73ms/step - accuracy: 0.2724 - loss: 4.1359
Epoch 11/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 80s 74ms/step - accuracy: 0.2876 - loss: 3.9476
Epoch 1

We will now define a function called `generate_slogan` which will generate a slogan by predicting one word at a time based on a given starting phrase (the `seed_text`). This function will do this using our trained model, `gen_model`.

Here is a breakdown of how the algorithm works:  

Let us assume the dictionary mapping words to unique indices, `tokenizer.word_index`, looks like this:

> `{'computer': 1, 'hardware': 2, 'taking': 3, 'care': 4, 'of': 5}`

If the model's predicted index for the next word is 3 (`predicted_index = 3`), the loop will:

> Check 'computer' (index 1) → No match  
> Check 'hardware' (index 2) → No match  
> Check 'taking' (index 3) → Match found!  
> Assign output_word = "taking" and exit the loop.  

The `output_word` will be appended to the `seed_text`, and the process will continue to add words to the `seed_text` until we have reached the maximum number of words **or** an invalid prediction occurs.  

Carefully follow the code below and complete the missing parts as guided by the comments.

In [104]:
def generate_slogan(seed_text, max_words=20):
    for _ in range(max_words):

        # Tokenising and padding seed_text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding="pre")

        # Use your trained model (gen_model) on token_list to predict the probability distribution of the next word over the vocabulary
        predictions = gen_model.predict(token_list)

        # From the predicted probabilities (stored in the variable predictions), identify the word index with the highest probability using np.argmax()
        predicted_index = np.argmax(predictions)

        output_word = None

        # Searching for the word that corresponds to the predicted index
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        # If no valid word is found, algorithm stops
        if output_word is None:
            break # out of main loop

        # If valid a word is found, we append the predicted word to seed_text and continue the process until max_words is reached or an invalid prediction occurs
        # Complete the code below to add a word to seed_text
        seed_text += " " + output_word

    return seed_text

## Training Data for Slogan Classifier

We will now prepare the data we will use to train our classifier. For our classifier, the inputs will come from the `processed_slogans` column of our DataFrame, `df`. The outputs will be the different industry categories under the `industry` column.

In the code cell below, extract the unique values from the `industry` column in the DataFrame and store these in a variable called **industries**.

In [105]:
# Gather unique industries
industries = df['industry'].unique()

print(f"Number of industries: {len(industries)}")
print("\nIndustries: ")
for industry in industries:
    print(industry)

Number of industries: 142

Industries: 
computer hardware
health, wellness and fitness
internet
financial services
mechanical or industrial engineering
marketing and advertising
hospital & health care
research
information technology and services
computer software
oil & energy
dairy
transportation/trucking/railroad
design
furniture
professional training & coaching
hospitality
textiles
food & beverages
management consulting
medical practice
accounting
performing arts
electrical/electronic manufacturing
higher education
outsourcing/offshoring
venture capital & private equity
writing and editing
mining & metals
construction
consumer electronics
retail
human resources
staffing and recruiting
farming
wholesale
events services
import and export
non-profit organization management
machinery
information services
biotechnology
philanthropy
law practice
real estate
graphic design
building materials
medical devices
consumer goods
automotive
plastics
business supplies and equipment
civil engineering

Create a dictionary called `industry_to_index` where each unique industry is mapped to a unique index starting from 0.

*Hint: Use the `enumerate()` function.*

In [106]:
# Create dictionary
industry_to_index = {industry: index for index, industry in enumerate(industries)}

print(industry_to_index)

{'computer hardware': 0, 'health, wellness and fitness': 1, 'internet': 2, 'financial services': 3, 'mechanical or industrial engineering': 4, 'marketing and advertising': 5, 'hospital & health care': 6, 'research': 7, 'information technology and services': 8, 'computer software': 9, 'oil & energy': 10, 'dairy': 11, 'transportation/trucking/railroad': 12, 'design': 13, 'furniture': 14, 'professional training & coaching': 15, 'hospitality': 16, 'textiles': 17, 'food & beverages': 18, 'management consulting': 19, 'medical practice': 20, 'accounting': 21, 'performing arts': 22, 'electrical/electronic manufacturing': 23, 'higher education': 24, 'outsourcing/offshoring': 25, 'venture capital & private equity': 26, 'writing and editing': 27, 'mining & metals': 28, 'construction': 29, 'consumer electronics': 30, 'retail': 31, 'human resources': 32, 'staffing and recruiting': 33, 'farming': 34, 'wholesale': 35, 'events services': 36, 'import and export': 37, 'non-profit organization management

Create a new column `industry_index` in your DataFrame by mapping the `industry` column to the indices using the `industry_to_index` dictionary.

*Hint: Use the  `map()` function.*

In [107]:
# Add new column with numerical industry codes
df['industry_index'] = df['industry'].map(industry_to_index)

# Check results
df[['industry', 'industry_index']].head()

,industry,industry_index
0,computer hardware,0
1,"health, wellness and fitness",1
2,internet,2
3,internet,2
4,financial services,3


Split the DataFrame `df` into training and testing sets, setting aside 20% of the data for the test set. Be sure to set the parameter `stratify=df["industry_index"]`. This ensures that both sets have the same proportion of each class (industry) as in the original dataset, resulting in balanced datasets. Call the training DataFrame `df_train` and the testing DataFrame `df_test`.

In [108]:
"""
  Ran into an issue where train_text_split will not wprk as expected due to 'health, wellness, and fitness' only
  having 1 entry. Removing industry th only 1 entry to move forward since the goal of the lesson is to utilize the
  stratify parameter. Model should still run optimally since a sample of less than 5 will not help the model learn
  patterns.

  Other possible solution is to drop the Stratify parameter from the train_test_split function but we may not get an
  equal representation of industries amongst test and training data due to 80/20 split.

"""

# Count industry index appearances
industry_counts = df['industry_index'].value_counts()

# Keep industries with industry_count >= 5
valid_industries = industry_counts[industry_counts >= 5].index
non_valid_industries = industry_counts[industry_counts < 5].index
print(f"Industries removed: {len(non_valid_industries)}")
for industry in non_valid_industries:
    print(industry)

# Filter dataframe to keep balid_industries
df = df[df['industry_index'].isin(valid_industries)]

# Split data into training (80%) and testing (20%)
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    stratify=df["industry_index"],
    random_state=42
)

print(f"\nTraining set size: {len(df_train)}")
print(f"Testing set size: {len(df_test)}")

Industries removed: 24
122
87
99
73
72
78
83
124
138
132
120
117
139
129
133
130
140
135
119
131
137
136
134
141

Training set size: 4232
Testing set size: 1058


Our classifier will use padded slogan sequences as inputs, similar to input sequences used for the slogan generator. The difference is we will not use sequences that get progressively longer, but instead we will use **complete slogans**. This is because our classifier does not need to learn how to predict what word comes next. It needs the full context of a slogan to learn how to accurately predict the industry.  

The next steps will walk you through how to create these sequences.  

We previously created and fitted a `Tokenizer` object called `tokenizer` while preparing data for the slogan generator. Now, we will reuse it to convert words into numerical indices.  

In the code cell below, use the `texts_to_sequences()` **method** of `tokenizer` to transform the `processed_slogan` column in **both** the `df_train` and `df_test` DataFrames into sequences of numerical indices. Store the results in variables named `X_train` and `X_test`.

In [109]:
# Convert slogan to numerical sequences
X_train = tokenizer.texts_to_sequences(df_train["processed_slogan"])
X_test = tokenizer.texts_to_sequences(df_test["processed_slogan"])

print(f"Number of training samples: {len(X_train)}")
print(f"Number of testing samples: {len(X_test)}")

Number of training samples: 4232
Number of testing samples: 1058


The slogan sequences are of varying lengths. We will need to pad them the same way we did to the input sequences for the slogan generator. The `pad_sequences()` function can ensure the sequences in `slogan_sequences` have the same length.  

In the code cell below, use the `pad_sequences()` function to standardise the `slogan_sequences` lengths. Set the `maxlen` parameter to `max_seq_len`, the `padding` parameter to 0, and assign the resulting padded sequences to the same variables, `X_train` and `X_test`.

In [110]:
# Makes sequences same length
X_train = pad_sequences(X_train, maxlen=max_seq_len, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_seq_len, padding='pre')

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")


X_train shape: (4232, 15)
X_test shape: (1058, 15)


We have successfully created training and testing inputs for our model. Now, we will create the outputs - industry categories.

 In the code cell that follows, use `tf.keras.utils.to_categorical()` to apply one-hot encoding to the `industry_index` column of **both** `df_train` and `df_test` DataFrames. Assign the results to a variables named `y_train` and `y_test`.

 *Hint: set the `num_classes` parameter to the total number of industries in the DataFrame. The `industries` variable can be used to find this value.*

In [111]:
# One-hot coding industry indicies
y_train = tf.keras.utils.to_categorical(df_train["industry_index"], num_classes=len(industries))
y_test = tf.keras.utils.to_categorical(df_test["industry_index"], num_classes=len(industries))

print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

y_train shape: (4232, 142)
y_test shape: (1058, 142)


## Slogan Classifier Architecture

Configure the LSTM classifier following these steps:  


1. Create a Sequential model:  
   Use `tf.keras.models.Sequential()` to create a sequential model. This model will consist of an embedding layer, two LSTM layers, and a dense output layer.

2. Add an embedding layer which will convert words into dense vector representations. Configure this layer with:
   > * `total_words` as the vocabulary size.
   > * 100 as the embedding dimension.
   > * `max_seq_len` as the `input_length` (this is the length of the slogans).

3. Add the first LSTM layer. Configure it with:
   > * 150 units.
   > * Set `return_sequences` to `True` to ensure the layer outputs sequences for the next LSTM layer.

4. Add the second LSTM layer which will process the output from the previous LSTM layer. Configure it with:
   > * 100 units.
   > * No need to set `return_sequences` here (it is the final LSTM layer).

5. Add the dense output layer which will classify the data into industries. Configure it with:
   > * The number of unique industries as the number of units.
   > * The `softmax` activation function to get probabilities for each class (industry).

6. Use `Sequential` to arrange all layers in the correct order and complete the architecture of the LSTM model called **class_model**.

In [112]:
# Create classification model

class_model = Sequential([
    # Indicies to vector
    Embedding(input_dim=total_words, output_dim=100, input_length=max_seq_len),

    # LSTM layer
    LSTM(150, return_sequences=True),

    # Second LSTM layer
    LSTM(100),

    # Predict industry
    Dense(len(industries), activation='softmax')
])

class_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In the code cell below, compile `class_model` using `categorical_crossentropy` loss, an Adam optimiser, and an appropriate metric of your choice.

In [113]:
# Classifier
class_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


## Slogan Classification & Evaluation

In the code cell that follows, fit the compiled model on the inputs and outputs, setting **the number of epochs to 50**.

In [114]:
# Train the classifier
history = class_model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
)


Epoch 1/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - accuracy: 0.0710 - loss: 4.5465 - val_accuracy: 0.0860 - val_loss: 4.2417
Epoch 2/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.0739 - loss: 4.2812 - val_accuracy: 0.0860 - val_loss: 4.2325
Epoch 3/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.0836 - loss: 4.2346 - val_accuracy: 0.1011 - val_loss: 4.0966
Epoch 4/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.1535 - loss: 3.8865 - val_accuracy: 0.1333 - val_loss: 3.9242
Epoch 5/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.2087 - loss: 3.4401 - val_accuracy: 0.1645 - val_loss: 3.8571
Epoch 6/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.3025 - loss: 2.9959 - val_accuracy: 0.1749 - val_loss: 3.8586
Epoch 7/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - accuracy: 0.3803 - loss: 2.6061 - val_accuracy: 0.1786 - val_loss: 3.9482
Epoch 8/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - accuracy: 0.4628 - loss: 2.2074 - val_

Evaluate the model using the testing set. Add a comment on the model's performance.

The accuracy of the model shows how often it predicts the industry from a slogan correctly. Based on my sources below, the industry standards are between 70% and 90% but anything over 70% is realistic.

Initially, the model was preducing a Test accuaracy of .19 and Test loss of 7.4 indicating that the model was more or less randomly guessing the data. After looking through notes, it appears that adding a batch will provide more stable training but it didn't improve the performance of the model. It appears that the model may not have enough samples per industry to learn off of to be able to acuratley predict the industry. I'm not sure if possibly increasing the data quality or model architecture. The other thought would be to increase the epochs to increase the accuracy of training, but it already takes an hour to train.

Resource: https://www.deepchecks.com/question/what-is-a-good-accuracy-score-in-machine-learning/#:~:text=Everything%20above%2070%25%20is%20acceptable,various%20businesses%20and%20sectors'%20needs.

In [116]:
# Evaluate model
loss, accuracy = class_model.evaluate(X_test, y_test)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1925 - loss: 7.2069
Test Loss: 7.4233
Test Accuracy: 0.1919


We will now define a function called `classify_slogan` which takes a slogan as input and predicts the industry it belongs to using the trained model, `class_model`.  

Carefully follow the code below and complete the missing parts (indicated by ellipses) as guided by the comments.

In [117]:
def classify_slogan(slogan):
    # Use the preprocess_text to clean the input slogan (we defined this function in the preprocessing section)
    slogan = preprocess_text(slogan)

    # Converting the slogan to a sequence of indices
    sequence = tokenizer.texts_to_sequences([slogan])

    # Pad the sequence using the pad_sequences() function
    padded_sequence = pad_sequences(sequence, maxlen=max_seq_len, padding='pre')

    # Pass padded_sequence into the class_model to get the predicted probabilities for each indistry
    prediction = class_model.predict(padded_sequence)

    # Use np.argmax() to get the index of the industry with the highest probability.
    # Note: np.argmax() returns a NumPy array, specifically a one-element array in this context.
    # Extract the single value from this array and assign it to predicted_index.
    predicted_index = np.argmax(prediction)

    # Return the predicted industry
    return industries[predicted_index]

## Combining the two models

Run the code cell below to combine the two models: we will first generate a slogan for a company in the "internet" industry, then pass the generated slogan to the slogan classifier to see if it correctly classifies it as internet.

In [118]:
industry = "internet"
generated_slogan = generate_slogan(industry)
predicted_industry = classify_slogan(generated_slogan)

print(f"Generated Slogan: {generated_slogan}")
print(f"Predicted Industry: {predicted_industry}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Generated Slogan: internet web design glasgow 19 sydney development designers firm in minutes self bags china simple bags day families soft soft trademark
Predicted Industry: renewables & environment


**Compare the results and comment on any differences you notice between the generated slogans and the classifier’s predictions in the markdown cell below.**

Based on the results, it appears that the generated slogan is long and repeats words when produced. The slogan itself is not cohesive or form a real world example of a slogan. Initially the slogan appears to be working but then more random and chaotic. The expected industry of 'internet' was not produced. Appears that both models need increased training and functionality. Our low test accuracy is why the classifier struggles with generating slogans.